In [1]:
"""
UCI adult dataset with Fairlearn mitigator.
"""

import sys

sys.path.append('../')

from fairlearn.reductions import DemographicParity
from sklearn.linear_model import LogisticRegression
from src.data.datasets import fetch_openml_dataset
from src.models.fairlearn_mitigators import ExpGradMitigator
from src.models.sklearn_estimators import SklearnClfs
from src.eval.fairness import eval_binary_clf_fairness
from src.models.fairlearn_mitigators import get_threshold_optim

In [2]:
LOGREG = "Logistic regression"
LOGREG_DICT = {
    LOGREG: LogisticRegression(max_iter=1000)
}
METRICS = {}

In [3]:
########################################################################
# Part 1 - ExpGradMitigator


def uci_adult_exp_grad_pipline(sensitive, clfs_dict, constraint=DemographicParity()):
    """Run the pipeline for UCI adult dataset.

    Args:
        sensitive (str): sensitive feature
        clfs_dict (dict): dictionary of classifiers
        mitigator_name (str): name of the mitigator

    Returns:
        dict: dictionary of fairness metrics
    """
    # Fetch dataset
    uci_adult = fetch_openml_dataset("UCIadult", sensitive)
    X = uci_adult["features"]
    y_true = uci_adult["labels"]
    sensitive_features = uci_adult["sensitive"]

    # Fit classifiers
    print("Fitting classifiers...")
    clfs = SklearnClfs(clfs_dict)
    clfs.fit_estimator_all(X, y_true)
    
    # Fit mitigator
    print("Fitting mitigators...")
    mitigators = ExpGradMitigator(clfs_dict, constraint)
    mitigators.fit_estimator_all(X, y_true, sensitive_features=sensitive_features)

    # Predict
    y_pred = clfs.predict_all(X)
    y_pred_mitigated = mitigators.predict_all(X)
    
    # Evaluate fairness
    print("Evaluating fairness...")
    fairness = {}
    fairness_mitigated = {}
    for type in y_pred:
        fairness[type] = eval_binary_clf_fairness(y_true, y_pred[type], sensitive_features)
        fairness_mitigated[type] = eval_binary_clf_fairness(y_true, y_pred_mitigated[type], sensitive_features)

    return fairness, fairness_mitigated

In [4]:
def test_uci_adult_exp_grad_pipline():
    """Test uci_adult_exp_grad_pipline function."""
    logreg_raw, logreg_expgrad_fair = uci_adult_exp_grad_pipline("sex", LOGREG_DICT)
    METRICS["raw"] = logreg_raw
    METRICS["expgrad"] = logreg_expgrad_fair

    stats_list = ["accuracy", "selection rate", "true positive rate", "false positive rate"]
    print(f"{LOGREG} - Raw:")
    print(METRICS["raw"][LOGREG].overall[stats_list])
    print(METRICS["raw"][LOGREG].by_group[stats_list])
    print()

    print(f"{LOGREG} - Exponentiated Gradient:")
    print(METRICS["expgrad"][LOGREG].overall[stats_list])
    print(METRICS["expgrad"][LOGREG].by_group[stats_list])

test_uci_adult_exp_grad_pipline()  # Takes ~ 40 seconds to run

Fitting classifiers...
Fitting mitigators...
Evaluating fairness...
Logistic regression - Raw:
accuracy               0.853528
selection rate         0.195713
true positive rate     0.602892
false positive rate    0.067636
dtype: object
        accuracy  selection rate  true positive rate  false positive rate
sex                                                                      
Female  0.927866        0.075037            0.513284             0.021285
Male    0.816662        0.255559            0.618875             0.097044

Logistic regression - Exponentiated Gradient:
accuracy               0.836698
selection rate         0.164142
true positive rate     0.501754
false positive rate    0.057946
dtype: object
        accuracy  selection rate  true positive rate  false positive rate
sex                                                                      
Female  0.901927        0.153718            0.754664             0.080011
Male    0.804349        0.169311            0.456644    

In [7]:
########################################################################
# Part 2 - ThresholdOptimizer


def uci_adult_thrshd_optim_pipeline(sensitive, clfs_dict):
    # Fetch dataset
    uci_adult = fetch_openml_dataset("UCIadult", sensitive)
    X = uci_adult["features"]
    y_true = uci_adult["labels"]
    sensitive_features = uci_adult["sensitive"]
    
    post_est_dict = get_threshold_optim(X, y_true, sensitive_features, clfs_dict)
    y_pred_mitigated = {}
    for mitigator_name in post_est_dict:
        y_pred_mitigated[mitigator_name] = post_est_dict[mitigator_name].predict(
            X, sensitive_features=sensitive_features
        )
    
    # Evaluate fairness
    print("Evaluating fairness...")
    fairness_mitigated = {}
    for type in y_pred_mitigated:
        fairness_mitigated[type] = eval_binary_clf_fairness(y_true,
                                                            y_pred_mitigated[type],
                                                            sensitive_features)
    return fairness_mitigated

In [8]:
def test_uci_adult_thrshd_optim_pipeline():
    """Test uci_adult_thrshd_optim_pipeline function."""
    logreg_th_optim = uci_adult_thrshd_optim_pipeline("sex", LOGREG_DICT)
    METRICS["th_optim"] = logreg_th_optim
    
    stats_list = ["accuracy", "selection rate", "true positive rate", "false positive rate"]
    print(f"{LOGREG} - Threshold Optimizer:")
    print(METRICS["th_optim"][LOGREG].overall[stats_list])
    print(METRICS["th_optim"][LOGREG].by_group[stats_list])


test_uci_adult_thrshd_optim_pipeline()  # Takes ~ 40 seconds to run

/Users/zeyuyang/opt/anaconda3/envs/fairml/lib/python3.9/site-packages/fairlearn/postprocessing/_threshold_optimizer.py:285: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(


Evaluating fairness...
Logistic regression - Threshold Optimizer:
accuracy               0.800602
selection rate         0.254945
true positive rate     0.616069
false positive rate    0.141354
dtype: object
        accuracy  selection rate  true positive rate  false positive rate
sex                                                                      
Female  0.768528        0.254632            0.605992             0.211537
Male    0.816508        0.255100            0.617867             0.096824
